In [1]:
import json

In [54]:
from collections import Counter

In [70]:
with open('data.json', encoding='utf-8') as fh:
    data = json.load(fh)



In [56]:
def unique_or_not(json):
    keys = list(json.keys())
    tables = []
    
    for key in keys:
        
        tables.append(json[key]['table'])
        
    if len(Counter(tables).values())>1:
        return False
    else:
        return True

    return 0


In [73]:
keywords = ["select","from","join","on","where"]
query = ""


if unique_or_not(data):
    query = query + str(keywords[0]) + " "
    for key in data.keys():
        if keys.index(key)>=len(keys)-1:
            query = query + str(data[key]['column']) + " "
        else:
            query = query + str(data[key]['column']) + ","
    
    query = query + str(keywords[1]) +" "+ str(data[key]['table']) + ";"
    
    print(query)

else:
    print("hi")
    query = query + str(keywords[0]) + " "
    
    
    
    
    
    
    

hi
